# Project on testing ML techniques to identify YSOs in Spitzer IRAC data

## Breanna Crompvoets and Samuel Fielder

## Project Summary and Goals
Young Stellar Objects (YSOs) are newly forming stars which are yet to begin burning. They are split into different classes depending on their dust/gas envelope to protostar ratio; Class 0 having the greatest envelope and Class III no longer having an envelope. Due to the different ratios of envelope to protostar, each class appears differently in spectroscopy; thus the difference in fluxes between Spitzer IRAC bands are able to determine which class the data comes from. This project will focus on using the same data as Cornu and Montillaud (2021; CM21) to classify data points into three classes: Class I, Class II, and Contaminants. Only these three classes out of the original 9 available are chosen as Class 0 is too dusty to detect, and Class III is difficult to distinguish from regular stars. Furthermore, the contaminating classes (galaxies, shocks, stars, and PAHs) are of less concern -- we would like the algorithms to focus on distinguishing Class I and Class II from the rest. The original paper uses a multi-layer perceptron (MLP) with one hidden layer (20 neurons). Their results are presented in the below table.

|Class | Recall | Precision |
| --- | --- | --- |
|1 | 94.0% | 79.1 %|
|2 | 96.7% | 90.6% | 
|Other | 98.7%| 99.8%| 

The data for this project was pulled from https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/647/A116. These data include columns for four Spitzer IRAC bands (3.6 $\mu m$, 4.5 $\mu m$, 5.8 $\mu m$, and 8 $\mu m$) fluxes and errors, as well as from one Spitzer MIPS band (24 $\mu m$), along with the target values as determined via a manual classification scheme and the predicted data from CM21. We will only be using the four IRAC bands and their associated errors, as the MIPS band does not provide data for most objects. We use the same target values as they do for accurate comparison. 

This project seeks to use a multitude of algorithms learned over the semester to measure their effectiveness and compare it to the recreated MLP of CM21. These algorithms include: GridSearch with an SVC, GridSearch with a Logistic Regressor, a Stacking Ensemble with an SVC and a Logistic Regressor, a Gradient Boosting ensemble, a Random Forest ensemble, and an XGBoost ensemble. We also created our own MLP based off of their prescription. The workload was split as follows: B. Crompvoets completed the data cleaning and all other algorithms besides the MLP, and S. Fielder completed an MLP close to that of CM21, as well as creating a custom data loader/split. They communicated together on the best hyper-parameters to test.


## Import Libraries and set global variables

In [1]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# classic ML libraries
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, recall_score, precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,  GridSearchCV
import xgboost as xgb

# custom made libraries
from custom_dataloader import replicate_data

/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# settings for confusion matrix plots and classification reports
cm_blues = plt.cm.Blues
custom_labs = ['Class 1', 'Class 2', 'Others']

# Classical ML Techniques

For each of the below algorithms we ran a gridsearch over a wide variety of hyperparameters. These parameter dictionaries are commented out in each cell, and we use the best parameters as a sample here to show how each algorithm performed. This section of the project was conducted by B. Crompvoets.

We conducted these fits for each of three data splits:
* "75/25" -- here the data is split into 75% training, 25% test set.
* "300s" -- here the data is split such that 5 out of the 7 subclasses, each has 300 members in the training set (2 do not have enough members). The test set size is the same as CM21. The data is again split into only 3 classes to train/test.
* "CM21" -- here the data is split with the exact same values as CM21 provide in their paper.

An example of a run with the 75/25 split is given below, with best results for each of the other runs commented out, as well as the parameter grid tested for each GridSearch. The results of all runs will be included at the end of this document.

## Loading Data Set

In [3]:
# data load
X = np.load("Input_Class_AllClasses_Sep.npy")
# Y = np.load("Target_Class_AllClasses_Sep.npy") # For original targets via Gutermuth 2009 Method
Y = np.load("Pred_Class_AllClasses_Sep.npy") # For predicted targets from CM21


# custom data loader to pull in custom sized data set
# use seed to get replicable results for now
seed_val = 1111

# the amounts below are how many of each class of object you want in the training set and validation set - leftover amounts given to testing set

# CM21 Split
amounts_train = [331,1141,231,529,27,70,1257]
amounts_val = [82, 531, 104, 278, 6, 17, 4359]

# 300s Split
# amounts_train = [300,300,300,300,27,70,300]
# amounts_val = [82, 531, 104, 278, 6, 17, 4359]

# 75/25 Split
# amounts_train = [311,1994,391,1043,25,66,21796] #75/25 train
# amounts_val = [103,665,130,348,9,22,5449] #75/25 val

# calling custom datagrabber here
inp_tr, tar_tr, inp_va, tar_va, inp_te, tar_te = replicate_data(X, Y, 'three', amounts_train, amounts_val, seed_val)

# scaling data according to training inputs
scaler_S = StandardScaler().fit(inp_tr)
inp_tr = scaler_S.transform(inp_tr)
inp_va = scaler_S.transform(inp_va)

# COMMENT NEXT LINE IF RUNNING 75/25 SPLIT
# inp_te = scaler_S.transform(inp_te) # Comment out for 75/25 split

# printouts for double checking all the sets and amounts
print('Sizes of Datasets : Inputs , Targets')
print('------------------------------------')
print(f'Training set: {inp_tr.shape} , {tar_tr.shape} \nValidation set: {inp_va.shape} , {tar_va.shape} \nTesting Set: {inp_te.shape}, {tar_te.shape}')
print('------------------------------------')


Sizes of Datasets : Inputs , Targets
------------------------------------
Training set: (1703, 8) , (1703,) 
Validation set: (717, 8) , (717,) 
Testing Set: (24483, 8), (24483,)
------------------------------------


In [4]:

def bootstrap_estimate_and_ci(estimator, X_tr, y_tr, X_va, y_va, scoring_func=None, random_seed=0, 
                               alpha=0.05, n_splits=200):
                        
    scores = []

    if scoring_func == accuracy_score:
        for n in range(0,n_splits):
            estimator.fit(X_tr, y_tr.ravel())  
            scores.append(scoring_func(y_va,estimator.predict(X_va)))
            # scores = list(map(list, zip(*scores)))
        estimate = np.mean(scores)
        lower_bound = np.percentile(scores, 100*(alpha/2))
        upper_bound = np.percentile(scores, 100*(1-alpha/2))
        stderr = np.std(scores)   

    else:
        for n in range(0,n_splits):
            estimator.fit(X_tr, y_tr.ravel())  
            scores.append(scoring_func(y_va,estimator.predict(X_va),average=None))   
            scores = list(map(list, zip(*scores)))
    
        estimate = [np.mean(scores[0]),np.mean(scores[1]),np.mean(scores[2])]
        lower_bound = [np.percentile(scores[0], 100*(alpha/2)),np.percentile(scores[1], 100*(alpha/2)),np.percentile(scores[2], 100*(alpha/2))]
        upper_bound = [np.percentile(scores[0], 100*(1-alpha/2)),np.percentile(scores[1], 100*(1-alpha/2)),np.percentile(scores[2], 100*(1-alpha/2))]
        stderr = [np.std(scores[0]),np.std(scores[1]),np.std(scores[2])]
    
    return estimate, lower_bound, upper_bound, stderr

In [5]:
f = open("PRAScores_CM21_C-targets.txt","w")
f.write("C-targets \n")
f.write("CM21 split \n")

# f.close()

12

## Logistic Regression


Specifying logistic regression
logreg = LogisticRegression()

hyperparameters tested over initially
param_grid = [{'penalty': ['l1'], 'max_iter': np.arange(300,1500,100),
        'solver': ['liblinear', 'saga'], 'tol': np.arange(0.0001,0.01,0.0005)},
        {'penalty': ['l2'], 'max_iter': np.arange(300,1500,100),
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'tol': np.arange(0.0001,0.01,0.0005)},
        {'penalty': ['elasticnet'], 'max_iter': np.arange(100,2000,100), 'l1_ratio': np.arange(0.1,1.,0.1),
        'solver': ['saga'], 'tol': np.arange(0.0001,0.01,0.0005)}]

grid = GridSearchCV(logreg,param_grid=param_grid, verbose=1)

Run the data through the grid to find optimal results
grid.fit(inp_tr, tar_tr.ravel())

In [6]:
%%time
# 75/25
# logreg = LogisticRegression('l1',max_iter=500,solver='saga',tol=0.0001)
# 300s
# logreg = LogisticRegression('l1',max_iter=300,solver='saga',tol=0.0016)
# CM21
logreg = LogisticRegression('l1',max_iter=600,solver='saga',tol=0.0001)
logreg.fit(inp_tr, tar_tr.ravel())

CPU times: user 224 ms, sys: 1.82 ms, total: 226 ms
Wall time: 226 ms


/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=600, penalty='l1', solver='saga')

In [7]:
est, low, up, stderr = bootstrap_estimate_and_ci(logreg, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=recall_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                              

/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which 

In [8]:
f.write("LR Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")
# f.close()


1

In [9]:
est, low, up, stderr = bootstrap_estimate_and_ci(logreg, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("LR Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(logreg, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("LR Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
# f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
# f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which 

1

## SVM

Hyperparameters tested over initially
param_grid = [{'kernel':['rbf','sigmoid','linear','poly'], 'gamma':['auto','scale'], 'C':np.arange(0.1,1.,0.1)}]

In [10]:
%%time
# Final hyperparameters
# 75/25
# svc = SVC(kernel='rbf',gamma='auto',C=0.9)
# 300s
# svc = SVC(kernel='linear',gamma='auto',C=0.8)
# CM21
svc = SVC(kernel='rbf',gamma='auto',C=0.9)

CPU times: user 15 µs, sys: 17 µs, total: 32 µs
Wall time: 35 µs


In [11]:
est, low, up, stderr = bootstrap_estimate_and_ci(svc, inp_tr, tar_tr.ravel(),  inp_va, tar_va, scoring_func=recall_score, random_seed=0, 
                              alpha=0.05, n_splits=200)

f.write("SVC Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(svc, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("SVC Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(svc, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                            
f.write("SVC Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
f.write("\n")

1

## SVM/LR Stacking Ensemble
No GridSearch, uses best pars as defined previously. Adding multiple SVC's does not improve results. 

In [12]:
%%time

# Specify Gradient Boost
# 75/25
estimators = [('svc', SVC(kernel='rbf',gamma='auto',C=0.9,random_state=42))]
# 300s
# estimators = [('svc', SVC(C=0.8, gamma='auto', kernel='linear',random_state=42))]
# CM21 
# estimators = [('svc', SVC(C=0.9, gamma='auto', kernel='rbf',random_state=42))]

# As the parameters for the Logistic Regression didn't change much, we use the best pars from the first trial.
stacl = StackingClassifier(estimators=estimators,
                           final_estimator=LogisticRegression(penalty = 'l1', max_iter = 500, solver ='saga', tol =0.0001))

# # Run the data through the stacking classifier
# stacl.fit(inp_tr, tar_tr.ravel())

# # Find the predicted values
# pred_tr = stacl.predict(inp_tr)
# pred_va = stacl.predict(inp_va)

CPU times: user 22 µs, sys: 9 µs, total: 31 µs
Wall time: 31.9 µs


In [13]:
est, low, up, stderr = bootstrap_estimate_and_ci(stacl, inp_tr, tar_tr.ravel(),  inp_va, tar_va, scoring_func=recall_score, random_seed=0, 
                              alpha=0.05, n_splits=200)

f.write("Stack Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(stacl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("Stack Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(stacl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                            
f.write("Stack Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
f.write("\n")

1

## Gradient Boosting

Hyperparameters tested over initially
param_grid = [{'n_estimators': np.arange(50,250,50),'subsample':[0.5,1.0],
              'criterion':['friedman_mse'],'n_iter_no_change':[5],'warm_start':[True,False],
              'max_depth':np.arange(1,11,2),'max_features': ['sqrt','log2']}]

In [14]:
%%time
# Final hyperparameters
# 75/25
# boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=9,max_features='log2',
#                 n_estimators=50,n_iter_no_change=5,subsample=1.0,warm_start=True)

# 300s
# boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=5,max_features='log2',
#                 n_estimators=150,n_iter_no_change=5,subsample=1.0,warm_start=False)

# CM21
boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=7,max_features='log2',
                n_estimators=200,n_iter_no_change=5,subsample=1.0,warm_start=True)

CPU times: user 18 µs, sys: 13 µs, total: 31 µs
Wall time: 31.9 µs


In [15]:
est, low, up, stderr = bootstrap_estimate_and_ci(boostcl, inp_tr, tar_tr.ravel(),  inp_va, tar_va, scoring_func=recall_score, random_seed=0, 
                              alpha=0.05, n_splits=200)

f.write("GB Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(boostcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("GB Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(boostcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                            
f.write("GB Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
f.write("\n")

1

### XGBoost

In [16]:
# Specify XGBoost
xgbcl = xgb.XGBClassifier()

# Hyperparameters tested over initially
# param_grid = [{'subsample':[0.5,1.0],'max_depth':np.arange(1,11,2),'sampling_method':['uniform']}]

# Final hyperparameters
# 75/25
# xgbcl = xgb.XGBClassifier(max_depth=9,sampling_method='uniform',subsample=0.5)
# 300s
# xgbcl = xgb.XGBClassifier(max_depth=7,sampling_method='uniform',subsample=0.5)
# CM21
xgbcl = xgb.XGBClassifier(max_depth=9,sampling_method='uniform',subsample=1.0)

In [17]:
est, low, up, stderr = bootstrap_estimate_and_ci(xgbcl, inp_tr, tar_tr.ravel(),  inp_va, tar_va, scoring_func=recall_score, random_seed=0, 
                              alpha=0.05, n_splits=200)

f.write("XGB Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(xgbcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("XGB Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(xgbcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                            
f.write("XGB Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
f.write("\n")

/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:16:52] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:16:52] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:20:25] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/breannacrompvoets/opt/anaconda3/envs/SF/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:20:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


1

## Random Forest

Hyperparameters tested over initially
param_grid = [{'class_weight': ['balanced_subsample','balanced'], 'n_estimators': np.arange(50,250,50),
        'criterion': ['gini', 'entropy'], 'max_features': ['sqrt','log2'], 'oob_score':[True,False]}]

In [18]:
# Final hyperparameters
# 75/25
# rfcl = RandomForestClassifier(class_weight='balanced_subsample',criterion='entropy',max_features='log2',n_estimators=150,oob_score=False)
# 300s
# rfcl = RandomForestClassifier(class_weight='balanced',criterion='entropy',max_features='log2',n_estimators=50,oob_score=False)
# CM21
rfcl = RandomForestClassifier(class_weight='balanced',criterion='entropy',max_features='log2',n_estimators=100,oob_score=True)

In [19]:
est, low, up, stderr = bootstrap_estimate_and_ci(rfcl, inp_tr, tar_tr.ravel(),  inp_va, tar_va, scoring_func=recall_score, random_seed=0, 
                              alpha=0.05, n_splits=200)

f.write("RF Recall\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(rfcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=precision_score, random_seed=0, 
                              alpha=0.05, n_splits=200)
                            
f.write("RF Precision\n")
f.write("{:.3f}".format(est[0])+" _{"+"{:.3f}".format(low[0])+"}^{"+"{:.3f}".format(up[0])+"} , "+"{:.3f}".format(stderr[0])+"//\n")
f.write("{:.3f}".format(est[1])+" _{"+"{:.3f}".format(low[1])+"}^{"+"{:.3f}".format(up[1])+"} , "+"{:.3f}".format(stderr[1])+"//\n")
f.write("{:.3f}".format(est[2])+" _{"+"{:.3f}".format(low[2])+"}^{"+"{:.3f}".format(up[2])+"} , "+"{:.3f}".format(stderr[2])+"//\n")
f.write("\n")

est, low, up, stderr = bootstrap_estimate_and_ci(rfcl, inp_tr, tar_tr.ravel(), inp_va, tar_va.ravel(), scoring_func=accuracy_score, random_seed=0, 
                               alpha=0.05, n_splits=200)
                            
f.write("RF Accuracy\n")
f.write("{:.3f}".format(est)+" _{"+"{:.3f}".format(low)+"}^{"+"{:.3f}".format(up)+"} , "+"{:.3f}".format(stderr)+"//\n")
f.write("\n")

1

In [20]:
f.close()

In [21]:
import os
# os.system('say "tadaaa your program has probably failed"')
os.system('say "beep"')

0

# Neural Network

In this section, we describe in which ways we replicated the CM21 paper, and the choices we made along the way to derive the best results we were able to achieve. This section was performed by S. Fielder.

All the training has been performed prior to this notebook, and the state of the network saved at the appropriate time. Below we will just import the state of the system described, and run the testing set through for evaluation, and metrics along with Confusion Matrixes will be outputted for each subsection.

Many of the functions called below are semi-custom made, and are found in the appropriate `.py` files in this directory. The majority come from `NN_Defs.py` where the model construction along with the training and validation functions are located. Additionally, our custom data split maker is found in `custom_dataloader.py`, in which we can build reproducable sets of training, validation and testing sets depending on the number of subclasses wanted in each set. In this example, we focus on the results achieved by using the CM21 data-split as performed using this loader.

Finally `network_runner.py` was the script used in order to train all of the networks, and appropriately print out the metrics, along with saving plots for both confusion matrixes and loss values. Some of these outputs are shown in the `Saved_Final_Data/` directory. The settings found therein will be imported below to load the state of the networks as mentioned above.

## Importing

In [22]:
# library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

# custom script inputs
from NN_Defs import get_n_params, train, validate, BaseMLP

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Running on : {device}')

datadir = 'Saved_Final_Data/'

Running on : cpu


## Data Loading

In [23]:
# data load
X = np.load("Input_Class_AllClasses_Sep.npy")
Y = np.load("Target_Class_AllClasses_Sep.npy")

# custom data loader to pull in custom sized data set
# use seed to get replicable results for now
seed_val = 1111

# the amounts below are how many of each class of object you want in the training set and validation set - leftover amounts given to testing set


# CM21 Split
# amounts_train = [331,1141,231,529,27,70,1257]
# amounts_val = [82, 531, 104, 278, 6, 17, 4359]

# 300s Split
amounts_train = [300,300,300,300,27,70,300]
amounts_val = [82, 531, 104, 278, 6, 17, 4359]

# # 75/25 Split
# amounts_train = [311,1994,391,1043,25,66,21796] #75/25 train
# amounts_val = [103,665,130,348,9,22,5449] #75/25 val


# calling custom datagrabber here
inp_tr, tar_tr, inp_va, tar_va, inp_te, tar_te = replicate_data(X, Y, 'three', amounts_train, amounts_val, seed_val)

# scaling data according to training inputs
scaler_S = StandardScaler().fit(inp_tr)
inp_tr = scaler_S.transform(inp_tr)
inp_va = scaler_S.transform(inp_va)
inp_te = scaler_S.transform(inp_te) # Comment out for 75/25 split

# printouts for double checking all the sets and amounts
print('Sizes of Datasets : Inputs , Targets')
print('------------------------------------')
print(f'Training set: {inp_tr.shape} , {tar_tr.shape} \nValidation set: {inp_va.shape} , {tar_va.shape} \nTesting Set: {inp_te.shape}, {tar_te.shape}')
print('------------------------------------')

Sizes of Datasets : Inputs , Targets
------------------------------------
Training set: (1597, 8) , (1597,) 
Validation set: (5377, 8) , (5377,) 
Testing Set: (19929, 8), (19929,)
------------------------------------


## Scaling, Conversions to Tensors, and DataLoader Creation

In [24]:
# creation of tensor instances
inp_tr = torch.tensor(inp_tr)
tar_tr = torch.tensor(tar_tr)
inp_va = torch.tensor(inp_va)
tar_va = torch.tensor(tar_va)
inp_te = torch.tensor(inp_te)
tar_te = torch.tensor(tar_te)

# pass tensors into TensorDataset instances
train_data = data_utils.TensorDataset(inp_tr, tar_tr)
val_data = data_utils.TensorDataset(inp_va, tar_va)
test_data = data_utils.TensorDataset(inp_te, tar_te)

# constructing data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=25, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=25, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=25, shuffle=True)

: 

: 

## Create Network Instance and Set Hyperparameters

The CM21 paper provided a good amount of detail about how the network was set up, including number of layers, number of neurons, how the data set was split for training and validation, etc. CM21 did mention that at every 3000 and 5000 epochs in training, they would check their metric to see if a reduction in learning rate would be useful, however they do not say conclusively if they implemented any learning rate scheduling.

We decided, as a starting point to implement the quoted learning rate of $4 \times10^{-5}$ statically throughout training, to see if decent results would emerge from a few thousand epochs of training.

Additionally, we would like to mention that after a stable network that produced decent results was built, going back and changing the batch size did not significantly affect the results, hence these will be kept at the constant size of 25 throughout the results presented here.

## Initial Results

In [ ]:
# create nn instance
BaseNN = BaseMLP(8, 20, 3)
# load in saved state of network
loadpath = datadir+'CSplit_LR4e-5_B25_E20k_M06_Settings'
BaseNN.load_state_dict(torch.load(loadpath, map_location=device))

# compute validation results
val_loss, val_accuracy, val_predictions, val_truth_values = validate(BaseNN, val_loader, device)

At the time of training, training and validation losses were also stored and plotted. This is loaded in manually here from `/Saved_Final_Data`.

<img src="Saved_Final_Data/CSplit_LR4e-5_B25_E20k_M06_loss.png" alt="Loss with 4e-5 Learning Rate" width="750"/>

As we can see from the above plot, the training and validation sets are still decreasing in value after 20000 epochs. We can additionally display our metrics as well as a Confusion Matrix.

In [ ]:
# print classification report
print(classification_report(val_truth_values, val_predictions, target_names=custom_labs))

# plotting Confusion Matrix and saving
fig, ax = plt.subplots(figsize=(5,5))
ConfusionMatrixDisplay.from_predictions(val_truth_values, val_predictions, ax = ax, normalize='true', cmap=cm_blues, display_labels=custom_labs, colorbar=False)
ax.set_title('Validation Set')
plt.tight_layout()
plt.show()

## A more aggressive approach

We found from above, that taking a static learning rate of very low value, yielded learning that was very slow. Of course when the learning rate is low, the step size for the SGD is so small that it will need quite a bit of time to yield necessary results that are comparable to the paper. From the confusion matrix above, we know that the model at it's early stages does a great job at classifying the "Others" class as well as the Class II objects, but the network is not yet picking up the differences between Class I and Class II objects.

What is needed, are larger learning rate steps by the optimizer, to speed up the process of finding a suitable minimum which produces the results closer to CM21's results. This **time to converge** is what we ended up tuning our network to produce.

Below, we will load in the network that was trained with a learning rate of $4 \times 10^{-2}$ for 10,000 epochs.

In [ ]:
# create nn instance
BaseNN = BaseMLP(8, 20, 3)
# load in saved state of network
# loadpath = datadir+'Final_CSplit_4e-2_Settings' #CM21 Split
loadpath = datadir+'Final_300s_Settings'
# loadpath = datadir+'Final_7525Split_Settings' # 75/25 Split

BaseNN.load_state_dict(torch.load(loadpath, map_location=device))

# compute validation results
val_loss, val_accuracy, val_predictions, val_truth_values = validate(BaseNN, val_loader, device)

Again, the loss plot below was constructed at our last epoch in this specific model loaded. This is loaded in manually here from `/Saved_Final_Data`.

<img src="Saved_Final_Data/Final_CSplit_4e-2_loss.png" alt="Loss with 4e-5 Learning Rate" width="750"/>

Surpringsly, we had not yet implemented any amount of learning rate scheduler, so to see the characteristic dip of a learning rate scheduler at around Epoch 1800 was surprising. This simply means that at that point in the model, the model fell into a minimum and plateaued from then on out.

In [ ]:
# print classification report
print(classification_report(val_truth_values, val_predictions, target_names=custom_labs))

# plotting Confusion Matrix and saving
fig, ax = plt.subplots(figsize=(5,5))
ConfusionMatrixDisplay.from_predictions(val_truth_values, val_predictions, ax = ax, normalize='true', cmap=cm_blues, display_labels=custom_labs, colorbar=False)
ax.set_title('Validation Set')
plt.tight_layout()
plt.show()

With only 10,000 epochs, and a learning rate that is much more aggressive, we already see the network perform quite well compared to the small learning rate model shown previously. It seems we do not only converge faster, but we also converge to a lower loss value, yielding high recall for our classes.

The largest change with the more aggressive learning rate, is that the network is now able to distinguish between Class I and Class II objects more effectively. Whereas prior our recall was 0% for Class I objects, that is now 87% recall, while training for only half the time.

## Can we be even more aggressive?

We then decided to run an even more aggressive learning rate, to see if we could find a deeper minimum, increasing our values for the metrics we focus on, mainly recall.

In [ ]:
# create nn instance
BaseNN = BaseMLP(8, 20, 3)
# load in saved state of network
loadpath = datadir+'Final_CSplit_4e-1_Settings'
BaseNN.load_state_dict(torch.load(loadpath, map_location=device))

# compute validation results
val_loss, val_accuracy, val_predictions, val_truth_values = validate(BaseNN, val_loader, device)

Again, the loss plot below was constructed at our last epoch in this specific model loaded. This is loaded in manually here from `/Saved_Final_Data`.

<img src="Saved_Final_Data/Final_CSplit_4e-1_loss.png" alt="Loss with 4e-5 Learning Rate" width="750"/>

In [ ]:
# print classification report
print(classification_report(val_truth_values, val_predictions, target_names=custom_labs))

# plotting Confusion Matrix and saving
fig, ax = plt.subplots(figsize=(5,5))
ConfusionMatrixDisplay.from_predictions(val_truth_values, val_predictions, ax = ax, normalize='true', cmap=cm_blues, display_labels=custom_labs, colorbar=False)
ax.set_title('Validation Set')
plt.tight_layout()
plt.show()

From the above results, we can see that implementing a learning rate that is too aggressive leads to convergence that is perhaps too quick, as we see the validation loss start to increase just past 3000 epochs. Overall, this network loses out to our previous value of learning rate, with a decrease in both recall for Class I and Class II objects.

## Final Implementation

To compare evenly between runs that have different learning rates, we of course have to run the networks with learning rates for much much longer. This was one of the feedbacks we were given following the presentation. We decided that we would run that initial network with the quoted learning rate in the paper, for a total of 500,000 epochs, to try and give the network a little more time to get it's metrics up.

In [ ]:
# create nn instance
BaseNN = BaseMLP(8, 20, 3)
# load in saved state of network
loadpath = datadir+'BIG_CSplit_LR4e-5_E500k_M06_Settings'
BaseNN.load_state_dict(torch.load(loadpath, map_location=device))

# compute validation results
val_loss, val_accuracy, val_predictions, val_truth_values = validate(BaseNN, val_loader, device)

<img src="Saved_Final_Data/BIG_CSplit_LR4e-5_E500k_M06_loss.png" alt="Loss with 4e-5 Learning Rate for 500k epochs" width="750"/>

Without much doubt, the network is still progressing towards lower loss values, even after 500,000 epochs. We do see an aggressive plateau after around 300,000 epochs, which could suggest that no amount of more training can help the network. Shown below in the metrics is the overall reasoning behind abandoning this approach altogether.

In [ ]:
# print classification report
print(classification_report(val_truth_values, val_predictions, target_names=custom_labs))

# plotting Confusion Matrix and saving
fig, ax = plt.subplots(figsize=(5,5))
ConfusionMatrixDisplay.from_predictions(val_truth_values, val_predictions, ax = ax, normalize='true', cmap=cm_blues, display_labels=custom_labs, colorbar=False)
ax.set_title('Validation Set')
plt.tight_layout()
plt.show()

As seen above, our recall metric has negligibly increased for our similar learning run with only 20,000 epochs. The network did improve significantly for recall in Class 2 objects, increasing from around 75% to 94%. This increase in recall was a direct result of lowering the amount of false negatives for Class 2 objects.

It may very well be that the network proposed by CM21 indeed used this technique, and we do note that results could be achieved with this method, granted enough time for the network to properly classify Class 1 objects from the Others class of objects.

# Results and Discussion

In the three images below we summarize the results of recall for each algorithm and our best performing MLP with each different data-split. We see that, if we are focused purely on achieving the best recall for Class I, then the SVM (with the "300's" split) is our best algorithm. However if we look across multiple classes, we find that the MLP (with CM21 split) does perform best, followed closely by XGBoost (also with CM21 split). 

These results indicate that classical ML methods are worth considering, as they give good performance given their simplicity in coding, and also in their implementation time for training (comparably negligible to the neural network). Overall, the justification to use the neural network approach for this problem is it's generalizability. At any point the network can be continued to train on newer data, and perhaps even make predictions on a dataset that is outside of the scope of the original dataset. The simplicity of the model itself, only having one hidden layer with a total of around 240 parameters, means that the network can easily be adapted to train further on an unrelated dataset, and perhaps yield to better predictions overall. We believe that if time is no constraint (which is rarely the case), we would choose the MLP for the overall great accuracy achieved, arriving the closest to the published results given by the CM21 Paper.

It is worth noting that CM21 used a simplified, observer-biased, labelling scheme in order to originally label the target data. This scheme and it's associated errors means there is non-negligible probability for falsely labelled data. It is quite likely that the errors in some of our methods are direct results of the incorrectly labelled targets. 

Furthermore, our network and algorithms only made use of eight data columns, not ten as CM21 did. These last two columns are important for separating Class I from reddenned Class II and would hence further aid in the training if they were available. As the data in these columns is sparse, we chose not to use them. To be specific, we would have to choose fill values for the missing entries in the dataset, and we were not confident that we could come up with a perscription that would justify filling in so many values.

Additionally, one approach that was not taken, would be artificial data augmentation. Since the data set is represented as wavelength measurements with their associated error, there could have been a case made for using some distribution and using small perturbations to create new data that would resemble the prior. This could have helped eleviate the large unbalanced dataset, allowing for the simply approach of splitting the dataset 75/25 to perhaps derive results that were "on par" with the other splitting techniques used.

Finally, CM21 do suggest that future work with this dataset could utilize their predicted results as the truth labels. This follows from a statement they made, that their network is more accurately predicting the target labels than the simplified labelling scheme originally did.

Recall Class I:

<img src="Saved_Final_Data/Recall_ClassI.png" alt="Recall Class I Results" width="1000"/>

Recall Class II:

<img src="Saved_Final_Data/Recall_Class2.png" alt="Recall Class II Results" width="1000"/>

Recall Class II:

<img src="Saved_Final_Data/Recall_Contaminants.png" alt="Recall Class Others Results" width="1000"/>